# 🎓 Notebook 11: Agentic RAG & Planning (ReAct)

Welcome to the **Pinnacle** of RAG Engineering. In this notebook, we transform our passive retrieval engine into an **Autonomous Agent**.

### What is an Agent?
Instead of a simple linear pipeline (`Question -> Search -> Answer`), an Agent follows a **ReAct** (Reason + Act) loop:
1. **Thought**: Analyze the question and decide what information is missing.
2. **Action**: Choose a tool (e.g., Vector Search, Web Search, Calculator).
3. **Observation**: Read the tool's output.
4. **Repeat**: Continue until a definitive answer can be synthesized.

### Learning Objectives:
1. Implement a manual ReAct loop from scratch.
2. Orchestrate multiple tools (Hybrid Search + Tavily Web Search).
3. Understand "Agentic Planning" vs. static routing.

In [1]:
import sys
import os

root = os.getcwd()
rag_root = None

while True:
    candidate = os.path.join(root, 'sprints', 'rag_engine', 'rag-engine-mini')
    if os.path.isdir(os.path.join(candidate, 'src')):
        rag_root = candidate
        break
    if os.path.basename(root) == 'rag-engine-mini' and os.path.isdir(os.path.join(root, 'src')):
        rag_root = root
        break
    parent = os.path.dirname(root)
    if parent == root:
        break
    root = parent

if rag_root:
    sys.path.insert(0, rag_root)


In [2]:
import json
from src.core.bootstrap import get_container
from src.application.use_cases.ask_question_hybrid import AskQuestionHybridUseCase

container = get_container()
use_case: AskQuestionHybridUseCase | None = container.get("ask_hybrid_use_case")
llm = container.get("llm")

if use_case is None:
    print("ask_hybrid_use_case not configured; skipping live workflow steps.")

## 1. Defining Tools

Our agent will have two primary tools: `internal_knowledge` (RAG) and `web_search` (Tavily).

In [3]:
def internal_knowledge_tool(query: str):
    print(f"[Tool] Searching internal RAG for: {query}")
    if use_case is None:
        return "RAG backend not configured; skipping retrieval."
    result = use_case.execute_retrieval_only(tenant_id="default", question=query)
    return "\n".join([getattr(c, "text", getattr(c, "content", "")) for c in result[:3]])


def web_search_tool(query: str):
    print(f"[Tool] Searching the web for: {query}")
    # Note: Requires TAVILY_API_KEY in .env
    from src.adapters.web_search.tavily_search import TavilySearch
    from src.core.config import settings
    searcher = TavilySearch(api_key=settings.tavily_api_key)
    return "\n".join([r.content for r in searcher.search(query)[:3]])


## 2. The ReAct Loop (The Brain)

We use the LLM to decide which tool to use. If the internal search fails or the question is about current events, it should "plan" to use the web.

In [4]:
import os

SYSTEM_PROMPT = """
You are an Autonomous Research Agent.
Available Tools:
1. internal_knowledge: Use for technical documentation, physics, or specific data.
2. web_search: Use for current events, news, or if internal search fails.

PROCESS:
THOUGHT: Explain why you are choosing a tool or finalizing the answer.
ACTION: tool_name(query)
OBSERVATION: Output from the tool.
FINAL ANSWER: Your final synthesis.
"""

def run_agent(question: str):
    prompt = f"{SYSTEM_PROMPT}\nQuestion: {question}\nTHOUGHT:"

    # For this demo, we do a single-step ReAct. In production, this would be a while-loop.
    if llm is None or not os.getenv("OPENAI_API_KEY"):
        return "LLM not configured; set OPENAI_API_KEY to run this demo."

    response = llm.generate(prompt)
    print(response)

    if "ACTION: internal_knowledge" in response:
        query = response.split("(")[1].split(")")[0]
        obs = internal_knowledge_tool(query)
        final = llm.generate(f"{prompt}\n{response}\nOBSERVATION: {obs}\nTHOUGHT: I have the data. FINAL ANSWER:")
        return final

    if "ACTION: web_search" in response:
        query = response.split("(")[1].split(")")[0]
        obs = web_search_tool(query)
        final = llm.generate(f"{prompt}\n{response}\nOBSERVATION: {obs}\nTHOUGHT: I have the search results. FINAL ANSWER:")
        return final

    return response


## 3. Live Execution

Observe how the agent "decides" based on the type of question.

In [5]:
print("--- Test 1: Internal Topic ---")
print(run_agent("What is the red planet?"))

print("\n--- Test 2: Current Events ---")
print(run_agent("What is the latest AI news from today?"))

--- Test 1: Internal Topic ---
LLM not configured; set OPENAI_API_KEY to run this demo.

--- Test 2: Current Events ---
LLM not configured; set OPENAI_API_KEY to run this demo.


## 4. Why this matters

Static RAG is limited by its retrieval algorithm. **Agentic RAG** creates a dynamic researcher that can solve multi-step problems, check its own errors, and explore new information sources autonomously.

**Congratulations!** You have mastered the most advanced pattern in current AI Engineering. 🚀